In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
!pip install -r requirements.txt

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 30.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 24.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 96.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 28.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.8/34.8 MB 45.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 53.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 89.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requi

In [3]:
!pipn install pandas

/bin/bash: line 1: pipn: command not found


In [4]:
import pandas as pd

def write_to_csv(question, answer, prediction, precision,recall,f1,hash):
    data = {'question': [question], 'answer': [answer], 'prediction': [prediction], 'precision': [precision],'recall':[recall],'f1':[f1],'hash':[hash]}
    df = pd.DataFrame(data)
    df.to_csv('evalute.csv', mode='a', header=False, index=False)


In [5]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
model_name='maxwell60701/genshin-impact-role-chat-model'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset
raw_datasets=load_dataset('maxwell60701/genshin-impact-role-chat-model')
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 4080
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 1020
    })
})

In [8]:
!pip install evaluate

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


In [10]:
!pip install bert_score

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


In [11]:
from evaluate import load
bertscore = load("bertscore")


In [12]:
for item in raw_datasets['train']:
   message=[item["messages"][0]]
   question=item["messages"][0]['content']
   answer=item["messages"][1]['content']
   print(question)
   print(answer)
   prompt=tokenizer.apply_chat_template(message,tokenize=True,add_generation_prompt=True,return_tensors="pt")
   prompt=prompt.to(model.device)
   output=model.generate(prompt,
                         temperature=0.1,
                         top_p=0.1, 
                         top_k=10,
                         max_new_tokens=512,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id)
   response=tokenizer.decode(output[0], skip_special_tokens=True)
   if "</think>" in response:
      prediction = response.split("</think>")[-1].strip()
   elif "<think>" in response:
      prediction = response.split("<think>")[-1].strip()
   else:
      prediction = response.strip()   
   print(prediction)
   predictions = [prediction]
   references = [answer]
   results = bertscore.compute(predictions=predictions, references=references, lang="zh", model_type="bert-base-chinese")
   print(results)
   precision=results['precision'][0]
   recall=results['recall'][0]
   f1=results['f1'][0]
   hash=results['hashcode']
   print(results['precision'][0])
   write_to_csv(question, answer, prediction, precision,recall,f1,hash)


妮露什么性别
妮露是女性
妮露是女性


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
你好,请问辛焱女士来自哪里?
辛焱来自璃月
辛焱来自璃月
辛焱来自璃月
{'precision': [0.8253868818283081], 'recall': [0.849582314491272], 'f1': [0.8373098373413086], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.8253868818283081
优菈女士掌握哪种元素力?
冰元素
优菈,冰元素
{'precision': [0.7329245209693909], 'recall': [0.9064151048660278], 'f1': [0.8104895949363708], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.7329245209693909
班尼特先生的元素是什么?
火元素
班尼特,班尼特元素
{'precision': [0.5901882648468018], 'recall': [0.7384580969810486], 'f1': [0.6560500860214233], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.5901882648468018
北斗女士是什么身份?
北斗是「南十字」的首领
北斗是「南十字」的首领
{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
琴女士是什么性别？
琴是女性
琴是女性
{'precision': [1.0], 're

烟绯使用火元素
烟绯用火元素
{'precision': [0.6886132955551147], 'recall': [0.8326339721679688], 'f1': [0.7538062334060669], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6886132955551147
林尼什么元素?
火元素
林尼,火元素
{'precision': [0.7282699346542358], 'recall': [0.9198611974716187], 'f1': [0.8129295706748962], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.7282699346542358
你好,九条裟罗女士使用哪种元素?
雷元素
九条裟罗使用雷元素
{'precision': [0.6099753379821777], 'recall': [0.8452364206314087], 'f1': [0.7085887789726257], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6099753379821777
达达利亚先生来自哪个国家?
达达利亚来自至冬
达达利亚来自至冬
{'precision': [0.9999998807907104], 'recall': [0.9999998807907104], 'f1': [0.9999998807907104], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.9999998807907104
珊瑚宫心海女士的元素是什么?
水元素
珊瑚宫心海，水元素
{'precision': [0.6441199779510498], 'recall': [0.8386667370796204], 'f1': [0.728630781173706], 'hashcode': 'be

In [13]:
for item in raw_datasets['test']:
   message=[item["messages"][0]]
   question=item["messages"][0]['content']
   answer=item["messages"][1]['content']
   print(question)
   print(answer)
   prompt=tokenizer.apply_chat_template(message,tokenize=True,add_generation_prompt=True,return_tensors="pt")
   prompt=prompt.to(model.device)
   output=model.generate(prompt,
                         temperature=0.1,
                         top_p=0.1, 
                         top_k=10,
                         max_new_tokens=512,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id)
   response=tokenizer.decode(output[0], skip_special_tokens=True)
   if "</think>" in response:
      prediction = response.split("</think>")[-1].strip()
   elif "<think>" in response:
      prediction = response.split("<think>")[-1].strip()
   else:
      prediction = response.strip()   
   print(prediction)
   predictions = [prediction]
   references = [answer]
   results = bertscore.compute(predictions=predictions, references=references, lang="zh", model_type="bert-base-chinese")
   print(results)
   precision=results['precision'][0]
   recall=results['recall'][0]
   f1=results['f1'][0]
   hash=results['hashcode']
   print(results['precision'][0])
   write_to_csv(question, answer, prediction, precision,recall,f1,hash)

班尼特用的是什么元素?
火元素
班尼特用火元素
{'precision': [0.6870261430740356], 'recall': [0.9119155406951904], 'f1': [0.7836556434631348], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6870261430740356
鹿野院平藏的元素属性是什么?
风元素
鹿野院平藏，风元素
{'precision': [0.6656675338745117], 'recall': [0.9033925533294678], 'f1': [0.7665213942527771], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6656675338745117
雷泽先生是什么性别？
雷泽是男性
雷泽是男性
{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
北斗使用什么元素力?
北斗使用雷元素
北斗使用雷元素
{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
神里绫华女士的职业是什么?
神里绫华是神里家族公主
神里绫华是鬼族女性，鬼道首领
{'precision': [0.7416591644287109], 'recall': [0.8152544498443604], 'f1': [0.7767174243927002], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.7416591644287109
你好,请问凯亚先生的性格特点是什么?
凯亚

诺艾尔是西风骑士团见习骑士，女仆
{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
菲米尼先生会用哪种元素?
水元素
菲米尼使用水元素
菲米尼,水元素
{'precision': [0.6493597030639648], 'recall': [0.8464993834495544], 'f1': [0.7349389791488647], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6493597030639648
菲米尼的性格特征是什么?
菲米尼,沉默而独自，怕人，爱水，专注于潜水技巧
菲米尼,沉默而独自，怕人，爱水，专业潜水员
{'precision': [0.9445036053657532], 'recall': [0.9174818396568298], 'f1': [0.9307966828346252], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.9445036053657532
辛焱女士,她是男孩还是女孩？
辛焱是女性
辛焱是女性
{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
云堇的性格特征是什么?
云堇,痴迷戏腔
云堇,痴迷戏腔
{'precision': [1.0000001192092896], 'recall': [1.0000001192092896], 'f1': [1.0000001192092896], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0000001192092896


烟绯是律法咨询师
{'precision': [1.0], 'recall': [1.0], 'f1': [1.0], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
1.0
诺艾尔会用哪种元素?
岩元素
诺艾尔使用冰元素
{'precision': [0.6215784549713135], 'recall': [0.7831536531448364], 'f1': [0.6930736899375916], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6215784549713135
妮露是男性还是女性？
妮露是女性
妮露是女性
妮露是女性
{'precision': [0.8366137742996216], 'recall': [0.8599392771720886], 'f1': [0.8481162190437317], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.8366137742996216
可莉是什么职务?
可莉是西风骑士团火花骑士，自认为有宝物
可莉,熟悉魔法师学科研究、专业学者
{'precision': [0.6226452589035034], 'recall': [0.6211052536964417], 'f1': [0.6218742728233337], 'hashcode': 'bert-base-chinese_L8_no-idf_version=0.3.12(hug_trans=4.46.3)'}
0.6226452589035034
枫原万叶有什么性格特征?
身经百战，有不为人所知的过往
枫原万叶,身经百战，有不为人所知的过往
{'precision': [0.8776211142539978], 'recall': [0.9728763103485107], 'f1': [0.9227970242500305], 'hashcode': 'bert-base-chinese_L8_no-idf_ve